### Labic Images Segmentation - Main Notebook

#### Importando módulos

In [ ]:
import labic_init
labic_init.labic_init()

In [ ]:
from labic_images_segmentation import Dataset, DataAugmentation, SegmentationModel, SaveReport, PredictImages, DiceCoef

In [ ]:
import tensorflow as tf
import json

#### Parâmetros

In [ ]:
with open("config.json", "r") as params:
    config = json.load(params)

In [ ]:
callbacks = []
if "early_stopping" in config["callback"]:
    callbacks.append(tf.keras.callbacks.EarlyStopping(monitor=config["monitor"], mode=config["mode_es"], \
                                                verbose=config["verbose_es"], patience=config["patience_es"]))
elif "reduce_lr_on_plateau" in config["callback"]:
    callbacks.append(tf.keras.callbacks.ReduceLROnPlateau(monitor=config["monitor"], factor=config["factor_lr"],
                              patience=config["patience_lr"], min_lr=["min_lr"]))
elif config["callback"] == None:
    callbacks = None

#### Treinamento

##### Carregando dataset

In [ ]:
imgs = Dataset(folder=config["dataset_folder"], norm_imgs_folder=config["norm_imgs_folder"], gt_folder=config["gt_folder"], \
                ORIGINAL_SIZE=config["ORIGINAL_SIZE"], NEW_SIZE=config["NEW_SIZE"], normalize=config["normalize"])

##### Dividindo dataset 

In [ ]:
imgs.split_dataset()

##### Data Augmentation

In [ ]:
data_aug = DataAugmentation(X_train=imgs.X_train, Y_train=imgs.Y_train, X_val=imgs.X_val, Y_val=imgs.Y_val, use_batch_size=config["batch_size"])

##### Treinamento da rede

In [ ]:
segment = SegmentationModel(N=imgs.X_train.shape[-1], segmentation_model=config["model"], backbone_name=config["backbone_name"],\
                             callback=callbacks, input_layer_shape=imgs.X_train.shape[1:], trainDS=data_aug.trainDS, valDS=data_aug.valDS, epochs=config["epochs"])

##### Salvando modelo e history

In [ ]:
save_report = SaveReport(model=segment.model, history=segment.history, folder_name=config["output_folder"], n_fold=config["n_fold"],\
                         use_batch_size=config["batch_size"], epochs=config["epochs"], exec_folder_name=config["exec_folder_name"])

#### Predição 

##### Carregando imagens de teste

In [ ]:
test_imgs = Dataset(folder=config["output_folder"], norm_imgs_folder=config["test_imgs_folder"], gt_folder=config["gt_test_folder"],\
                     ORIGINAL_SIZE=config["ORIGINAL_SIZE"], NEW_SIZE=config["NEW_SIZE"], normalize=config["normalize"])

#### Segmentando imagens de teste

In [ ]:
prediction = PredictImages(test_images=test_imgs, n_fold_folder_name=save_report.n_fold_folder_name, \
                           model_name=save_report.model_name, use_batch_size=config["batch_size"], img_shape=test_imgs.img_shape)

##### Salvando DSC

In [ ]:
dice = DiceCoef(gt_imgs=test_imgs.Y, pred_folder=prediction.n_fold_folder_name+"/outputs_prod/*", new_size=config["NEW_SIZE"])
n_fold = config["n_fold"]
dice.save_dice(f"{save_report.n_fold_folder_name}/dice_fold_{n_fold}.txt")

##### DSC final (n-folds)

In [ ]:
dice.generate_csv_dice(save_report=save_report, n_all_folders = config["n_fold"], title=config["title"])

##### Gráficos - Performance do Treino

In [ ]:
dice.generate_graphic(segment=segment, save_report=save_report, epochs=config["epochs"], graphic_type="iou_score")

In [ ]:
dice.generate_graphic(segment=segment, save_report=save_report, epochs=config["epochs"], graphic_type="loss")

##### DSC final (n-folds)

In [ ]:
dice.generate_csv_dice(save_report=save_report, n_all_folders = config["n_fold"], title=config["title"])

In [ ]:
dice.df